# Introdução

Este notebook visa apresentar o desenvolvimento realizado pelo Grupo 3 - composto por Tiago Sousa (20735) , Rodrigo Castro (23143), Rogério Gomes (27216), Paulo Costa (29851) e Laís Carvalho (51067) - no âmbito do trabalho prático “Project 01” da disciplina de Inteligêncial Artificial.

O projeto tem como objetivo a criação de um agente inteligente capaz de funcionar como gestor de horários de aulas para cursos de graduação numa instituição de ensino superior, considerando todas as restrições e condições que possam existir no processo de alocação de horas.

O problema de agendamento é resolvido utilizando programção de restrições (CSP).


In [6]:
# Install contraint library
!pip install python-constraint

# Import contraint library
from constraint import *


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\Paulo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## Variáveis e Domínio

In [7]:
# instatiate the problem
problem = Problem()

# add variables
problem.addVariable('uc11_t01_aula01', range(1,20))
problem.addVariable('uc11_t01_aula02', range(1,20))

problem.addVariable('uc12_t01_aula01', range(1,12))
problem.addVariable('uc12_t01_aula02', range(1,12))

problem.addVariable('uc13_t01_aula01', range(5,20))
problem.addVariable('uc13_t01_aula02', range(5,20))

problem.addVariable('uc14_t01_aula01', range(5,20))
problem.addVariable('uc14_t01_aula02', range(5,20))

problem.addVariable('uc15_t01_aula01', list(chain(range(1, 9), range(13, 17))))
problem.addVariable('uc15_t01_aula02', list(chain(range(1, 9), range(13, 17))))

problem.addVariable('uc21_t02_aula01', range(1,20))
problem.addVariable('uc21_t02_aula02', range(1,20))

problem.addVariable('uc22_t02_aula01', range(1,20))
problem.addVariable('uc22_t02_aula02', range(1,20))

problem.addVariable('uc23_t02_aula01', range(1,12))
problem.addVariable('uc23_t02_aula02', range(1,12))

problem.addVariable('uc24_t02_aula01', range(5,20))
problem.addVariable('uc24_t02_aula02', range(5,20))

problem.addVariable('uc25_t02_aula01', list(chain(range(1, 9), range(13, 17))))
problem.addVariable('uc25_t02_aula02', list(chain(range(1, 9), range(13, 17))))

problem.addVariable('uc31_t03_aula01', range(1,20))
problem.addVariable('uc31_t03_aula02', range(1,20))

problem.addVariable('uc32_t03_aula01', range(1,12))
problem.addVariable('uc32_t03_aula02', range(1,12))

problem.addVariable('uc33_t03_aula01', range(5,20))
problem.addVariable('uc33_t03_aula02', range(5,20))

problem.addVariable('uc34_t03_aula01', list(chain(range(1, 9), range(13, 17))))
problem.addVariable('uc34_t03_aula02', list(chain(range(1, 9), range(13, 17))))

problem.addVariable('uc35_t03_aula01', list(chain(range(1, 9), range(13, 17))))
problem.addVariable('uc35_t03_aula02', list(chain(range(1, 9), range(13, 17))))


# Restrições 

 --- FUNÇÕES AUXILIARES ---

In [8]:
from constraint import *
from itertools import chain

def get_day(slot):
    """Retorna o dia correspondente (1–5) para o slot (1–20)."""
    return (slot - 1) // 4 + 1

def get_block(slot):
    """Retorna o bloco (1–4) dentro do dia."""
    return (slot - 1) % 4 + 1

 --- RESTRIÇÕES ---

 1- Professor não pode ter 2 aulas ao mesmo tempo

In [9]:
teacher_courses = {
    "jo":  ["uc11_t01", "uc21_t02", "uc22_t02", "uc31_t03"],
    "mike":["uc12_t01", "uc23_t02", "uc32_t03"],
    "rob": ["uc13_t01", "uc14_t01", "uc24_t02", "uc33_t03"],
    "sue": ["uc15_t01", "uc25_t02", "uc34_t03", "uc35_t03"]
}

def teacher_conflict(a, b):
    return a != b  # não pode ter o mesmo slot

for teacher, cursos in teacher_courses.items():
    for i in range(len(cursos)):
        for j in range(i + 1, len(cursos)):
            problem.addConstraint(teacher_conflict, [f"{cursos[i]}_aula01", f"{cursos[j]}_aula01"])
            problem.addConstraint(teacher_conflict, [f"{cursos[i]}_aula01", f"{cursos[j]}_aula02"])
            problem.addConstraint(teacher_conflict, [f"{cursos[i]}_aula02", f"{cursos[j]}_aula01"])
            problem.addConstraint(teacher_conflict, [f"{cursos[i]}_aula02", f"{cursos[j]}_aula02"])



2- As duas aulas do mesmo curso não podem ser no mesmo slot

In [10]:

def different_times(a, b):
    return a != b

for c in [
    "uc11_t01", "uc12_t01", "uc13_t01", "uc14_t01", "uc15_t01",
    "uc21_t02", "uc22_t02", "uc23_t02", "uc24_t02", "uc25_t02",
    "uc31_t03", "uc32_t03", "uc33_t03", "uc34_t03", "uc35_t03"
]:
    problem.addConstraint(different_times, [f"{c}_aula01", f"{c}_aula02"])



3- As duas aulas da mesma cadeira devem ser em dias diferentes

In [11]:

def different_days(a, b):
    return get_day(a) != get_day(b)

for c in [
    "uc11_t01", "uc12_t01", "uc13_t01", "uc14_t01", "uc15_t01",
    "uc21_t02", "uc22_t02", "uc23_t02", "uc24_t02", "uc25_t02",
    "uc31_t03", "uc32_t03", "uc33_t03", "uc34_t03", "uc35_t03"
]:
    problem.addConstraint(different_days, [f"{c}_aula01", f"{c}_aula02"])



4- Cada turma deve ter aulas distribuídas por no máximo 4 dias

In [12]:

turmas = {
    "t01": ["uc11_t01", "uc12_t01", "uc13_t01", "uc14_t01", "uc15_t01"],
    "t02": ["uc21_t02", "uc22_t02", "uc23_t02", "uc24_t02", "uc25_t02"],
    "t03": ["uc31_t03", "uc32_t03", "uc33_t03", "uc34_t03", "uc35_t03"]
}

def max_four_days(*args):
    dias = {get_day(x) for x in args}
    return len(dias) <= 4

for t, cursos in turmas.items():
    all_lessons = [f"{c}_aula01" for c in cursos] + [f"{c}_aula02" for c in cursos]
    problem.addConstraint(max_four_days, all_lessons)




5- As aulas da mesma turma no mesmo dia devem ser em blocos consecutivos

In [13]:

def consecutive_blocks(*args):
    aulas_por_dia = {}
    for a in args:
        dia = get_day(a)
        bloco = get_block(a)
        aulas_por_dia.setdefault(dia, []).append(bloco)
    for blocos in aulas_por_dia.values():
        blocos.sort()
        for i in range(1, len(blocos)):
            if blocos[i] - blocos[i - 1] > 1:
                return False
    return True

for t, cursos in turmas.items():
    all_lessons = [f"{c}_aula01" for c in cursos] + [f"{c}_aula02" for c in cursos]
    problem.addConstraint(consecutive_blocks, all_lessons)


# 6️⃣ (Opcional Soft) Minimizar número de salas diferentes por turma
# Aqui simulamos como "máx. 3 dias distintos de slots" → aproximado
# Se fores adicionar salas depois, podes trocar esta função.



# Solução do problema

In [14]:

# --- EXECUÇÃO ---
solution = problem.getSolution()
print("\n--- Solução encontrada ---")
for var, slot in sorted(solution.items()):
    dia = (slot - 1)//4 + 1
    bloco = (slot - 1)%4 + 1
    print(f"{var}: Dia {dia}, Bloco {bloco}")


--- Solução encontrada ---
uc11_t01_aula01: Dia 5, Bloco 3
uc11_t01_aula02: Dia 4, Bloco 4
uc12_t01_aula01: Dia 3, Bloco 3
uc12_t01_aula02: Dia 2, Bloco 4
uc13_t01_aula01: Dia 5, Bloco 3
uc13_t01_aula02: Dia 4, Bloco 4
uc14_t01_aula01: Dia 5, Bloco 2
uc14_t01_aula02: Dia 4, Bloco 3
uc15_t01_aula01: Dia 4, Bloco 4
uc15_t01_aula02: Dia 2, Bloco 4
uc21_t02_aula01: Dia 5, Bloco 2
uc21_t02_aula02: Dia 4, Bloco 3
uc22_t02_aula01: Dia 5, Bloco 1
uc22_t02_aula02: Dia 4, Bloco 2
uc23_t02_aula01: Dia 3, Bloco 2
uc23_t02_aula02: Dia 2, Bloco 3
uc24_t02_aula01: Dia 5, Bloco 1
uc24_t02_aula02: Dia 4, Bloco 2
uc25_t02_aula01: Dia 4, Bloco 3
uc25_t02_aula02: Dia 2, Bloco 3
uc31_t03_aula01: Dia 4, Bloco 1
uc31_t03_aula02: Dia 3, Bloco 4
uc32_t03_aula01: Dia 2, Bloco 2
uc32_t03_aula02: Dia 1, Bloco 4
uc33_t03_aula01: Dia 4, Bloco 1
uc33_t03_aula02: Dia 3, Bloco 4
uc34_t03_aula01: Dia 4, Bloco 2
uc34_t03_aula02: Dia 2, Bloco 2
uc35_t03_aula01: Dia 4, Bloco 1
uc35_t03_aula02: Dia 2, Bloco 1
